In [1]:
## Building out scraper for nfl team stats
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import time

# Generating a range of ints by 100 to append to url at the end
# to navigate between pages
leaf = range(0, 9100, 100)

## root url below:
base_url = ('http://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=1999&year_max=2016&game_type=&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&game_day_of_week=&game_time=&time_zone=&surface=&roof=&temperature=&temperature_gtlt=lt&game_location=&game_result=&overtime=&league_id=NFL&team_id=&opp_id=&team_div_id=&opp_div_id=&team_conf_id=&opp_conf_id=&date_from=&date_to=&team_off_scheme=&opp_off_scheme=&team_def_align=&opp_def_align=&stadium_id=&c1stat=points&c1comp=gt&c1val=&c2stat=tot_yds&c2comp=gt&c2val=&c3stat=pass_cmp_opp&c3comp=gt&c3val=&c4stat=rush_att_opp&c4comp=gt&c4val=&c5comp=&c5gtlt=lt&c6mult=1.0&c6comp=&order_by=game_date&order_by_asc=&matching_games=1&conference_game=&division_game=&tm_is_playoff=&opp_is_playoff=&tm_is_winning=&opp_is_winning=&tm_scored_first=&tm_led=&tm_trailed=&tm_won_toss=&offset=')

## this will loop through the leaf list and creat my entire list of urls
## and append to the empty list "url_list"
url_list = []
for i in leaf:
    url_list.append(base_url + str(i))


In [2]:
## I am now creating a test link list so that I can work through the code
## and debug while working with a less extensive dataset

test_urls = url_list[0:1]

## as you can see below, my test_urls list is composed of only first link
print test_urls

['http://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=1999&year_max=2016&game_type=&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&game_day_of_week=&game_time=&time_zone=&surface=&roof=&temperature=&temperature_gtlt=lt&game_location=&game_result=&overtime=&league_id=NFL&team_id=&opp_id=&team_div_id=&opp_div_id=&team_conf_id=&opp_conf_id=&date_from=&date_to=&team_off_scheme=&opp_off_scheme=&team_def_align=&opp_def_align=&stadium_id=&c1stat=points&c1comp=gt&c1val=&c2stat=tot_yds&c2comp=gt&c2val=&c3stat=pass_cmp_opp&c3comp=gt&c3val=&c4stat=rush_att_opp&c4comp=gt&c4val=&c5comp=&c5gtlt=lt&c6mult=1.0&c6comp=&order_by=game_date&order_by_asc=&matching_games=1&conference_game=&division_game=&tm_is_playoff=&opp_is_playoff=&tm_is_winning=&opp_is_winning=&tm_scored_first=&tm_led=&tm_trailed=&tm_won_toss=&offset=0']


In [3]:
headers = []

html = urllib.urlopen(test_urls[0])
soup = BeautifulSoup(html, 'html.parser')

tab_header = soup.findAll('th')
for i in tab_header:
    headers.append(i.renderContents())
print len(headers)

## because of some extra headers mixed in, the actual column headers
## that we want are interspersed in the following range
print headers[6:42]

## these names are a bit confusing due to the organization in the table
## so they will probably just be manually recorded later on

190
['Tm', 'Year', 'Date', 'Time', 'LTime', '', 'Opp', 'Week', 'G#', 'Day', 'Result', 'OT', 'PF', 'PA', 'PD', 'PC', 'Tot', 'Ply', 'Y/P', 'DPly', 'DY/P', 'TO', 'ToP', 'Time', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'Int', 'Sk', 'Yds', 'Att', 'Yds', 'Y/A', 'TD']


In [4]:
# ## I am now going to work through my test url list to try to capture the 
# ## column headers to store for later use

# col_headers = []
# for link in test_urls:
#     html = urllib.urlopen(link).read()
#     soup = BeautifulSoup(html, 'html.parser')
#     column_head = soup.findAll('th')
#     for i in column_head:
#         col_headers.append(i.renderContents())
    
    
#     #col_name = soup.findAll('th', {'class':'tooltip sort_default_asc'})
# #print col_name


In [5]:
## The below code is an effort to scrape the stats for each game since 1999

## I am manually recording a list of column names and changing them to
## more easily interpretable names
team_data_cols = ['rk', 'team', 'year', 'date', 'east_time', 'loc_time', 'blank_@',
                 'opp', 'week', 'team_game#', 'day', 'result', 'ot', 'pf', 'pa', 'pdiff',
                 'pcomb', 'tot_yds', 'off_plys', 'off_yds/ply', 'def_plys', 'def_yds/ply', 'to_lost',
                  'off_time_poss','game_duration', 'opp_completions', 'opp_pass_att', 'opp_comp_perc',
                 'opp_pass_yds', 'opp_pass_tds', 'opp_int_thrown', 'opp_sacks_taken', 'opp_sacks_yds_lost',
                 'opp_rush_atts', 'opp_rush_yds', 'opp_rush_yds/att', 'opp_rush_td']

## redefining our html and soup just to make sure I am referencing the
## correct link and soup
html = urllib.urlopen(test_urls[0])
soup = BeautifulSoup(html, 'html.parser')

## creating an empty list to append the data to
data_points = []

## creating variable that consists of the body of webpage we are interested
## in combing through
body = soup.findAll('tbody')

## this variable creates a mass of the individual rows of the data to
## work our way through
indiv_rows = body[0].findAll('td')

## i will loop through each row to strip out the data
for row in indiv_rows:
    # the line below redefines my soup as the contents of each individual row
    inner_soup = BeautifulSoup(row.renderContents(), 'html.parser')
    # this adds the data to the empty list
    # the .text function strips hyperlinks and returns the text value only
    data_points.append(inner_soup.text)


In [6]:
## since data points is just one long list, i need to break it up into
## individual chunks so it can be added to our dataframe
chunks = [data_points[x:x+37] for x in range(0, len(data_points), 37)]

## Here I create a dataframe from the newly formed chunks and have
## the newly defined columns as my column names
team_df = pd.DataFrame(chunks, columns = team_data_cols)
team_df

,rk,team,year,date,east_time,loc_time,blank_@,opp,week,team_game#,...,opp_comp_perc,opp_pass_yds,opp_pass_tds,opp_int_thrown,opp_sacks_taken,opp_sacks_yds_lost,opp_rush_atts,opp_rush_yds,opp_rush_yds/att,opp_rush_td
0,1,CAR,2015,2016-02-07,6:39,3:39,@,DEN,21,19,...,56.5,104,0,1,5,37,28,90,3.21,1
1,2,DEN,2015,2016-02-07,6:39,3:39,,CAR,21,19,...,43.9,197,0,1,7,68,27,118,4.37,1
2,3,CAR,2015,2016-01-24,6:42,6:42,,ARI,20,18,...,56.1,227,1,4,3,8,16,60,3.75,1
3,4,ARI,2015,2016-01-24,6:42,6:42,@,CAR,20,18,...,67.9,324,2,1,1,11,37,152,4.11,3
4,5,DEN,2015,2016-01-24,3:05,1:05,,NWE,20,18,...,48.2,292,1,2,4,18,17,44,2.59,1
5,6,NWE,2015,2016-01-24,3:05,1:05,@,DEN,20,18,...,53.1,145,2,0,3,31,30,99,3.30,0
6,7,CAR,2015,2016-01-17,1:06,1:06,,SEA,19,17,...,64.6,325,3,2,5,41,12,78,6.50,0
7,8,DEN,2015,2016-01-17,4:40,2:40,,PIT,19,17,...,64.9,311,0,0,3,28,19,85,4.47,1
8,9,PIT,2015,2016-01-17,4:40,2:40,@,DEN,19,18,...,56.8,215,0,0,1,7,33,109,3.30,1
9,10,SEA,2015,2016-01-17,1:06,1:06,@,CAR,19,18,...,72.7,151,1,0,1,10,41,144,3.51,2


In [ ]:
### I am going to consolidate all the above code and unleash on all
### of the webpages we are going for in order to scrape all the data

## i am also importing time to put a sleep time in my loop
import time

leaf = range(0, 9100, 100)

base_url = ('http://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=1999&year_max=2016&game_type=&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&game_day_of_week=&game_time=&time_zone=&surface=&roof=&temperature=&temperature_gtlt=lt&game_location=&game_result=&overtime=&league_id=NFL&team_id=&opp_id=&team_div_id=&opp_div_id=&team_conf_id=&opp_conf_id=&date_from=&date_to=&team_off_scheme=&opp_off_scheme=&team_def_align=&opp_def_align=&stadium_id=&c1stat=points&c1comp=gt&c1val=&c2stat=tot_yds&c2comp=gt&c2val=&c3stat=pass_cmp_opp&c3comp=gt&c3val=&c4stat=rush_att_opp&c4comp=gt&c4val=&c5comp=&c5gtlt=lt&c6mult=1.0&c6comp=&order_by=game_date&order_by_asc=&matching_games=1&conference_game=&division_game=&tm_is_playoff=&opp_is_playoff=&tm_is_winning=&opp_is_winning=&tm_scored_first=&tm_led=&tm_trailed=&tm_won_toss=&offset=')

url_list = []
for i in leaf:
    url_list.append(base_url + str(i))
    
team_data_cols = ['rk', 'team', 'year', 'date', 'east_time', 'loc_time', 'blank_@',
                 'opp', 'week', 'team_game#', 'day', 'result', 'ot', 'pf', 'pa', 'pdiff',
                 'pcomb', 'tot_yds', 'off_plys', 'off_yds/ply', 'def_plys', 'def_yds/ply', 'to_lost',
                  'off_time_poss','game_duration', 'opp_completions', 'opp_pass_att', 'opp_comp_perc',
                 'opp_pass_yds', 'opp_pass_tds', 'opp_int_thrown', 'opp_sacks_taken', 'opp_sacks_yds_lost',
                 'opp_rush_atts', 'opp_rush_yds', 'opp_rush_yds/att', 'opp_rush_td']

data_points = []

body = soup.findAll('tbody')

indiv_rows = body[0].findAll('td')

## this initial loop has been added to circulate through all the links collected
# also adding count to check my status while conducting the loop
count = 0
for i in url_list:
    html = urllib.urlopen(i)
    soup = BeautifulSoup(html, 'html.parser')
    for row in indiv_rows:
        inner_soup = BeautifulSoup(row.renderContents(), 'html.parser')
        data_points.append(inner_soup.text)
    ## adding a sleep time to not ping website too frequently
    count += 1
    print "you have completed this many loops:  %d" % count
    time.sleep(1)

chunks = [data_points[x:x+37] for x in range(0, len(data_points), 37)]

team_df = pd.DataFrame(chunks, columns = team_data_cols)
team_df

you have completed this many loops:  1
you have completed this many loops:  2
you have completed this many loops:  3
you have completed this many loops:  4
you have completed this many loops:  5
you have completed this many loops:  6
you have completed this many loops:  7
you have completed this many loops:  8
you have completed this many loops:  9
you have completed this many loops:  10
you have completed this many loops:  11
you have completed this many loops:  12
you have completed this many loops:  13
you have completed this many loops:  14
you have completed this many loops:  15
you have completed this many loops:  16
you have completed this many loops:  17
you have completed this many loops:  18
you have completed this many loops:  19
you have completed this many loops:  20
you have completed this many loops:  21
you have completed this many loops:  22
you have completed this many loops:  23
you have completed this many loops:  24
you have completed this many loops:  25
